# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "human_promoters_flanking_ensembl_regulatory"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_promoters_flanking_ensembl_regulatory 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

2022-02-14 12:18:14.111293: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-14 12:18:14.111326: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-14 12:18:15.673711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-14 12:18:15.673762: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-14 12:18:15.673778: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (katarina-lenovo): /proc/driver/nvidia/version does not exist
2022-02-14 12:18:15.674012: I tensorflow/core/platform/cpu_fe

In [5]:
info(DATASET)

## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 198254 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
# vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

['', '[UNK]', 't', 'a', 'g', 'c', 'n']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
3098/3098 [==============================] - 505s 162ms/step - loss: 0.6257 - binary_accuracy: 0.6544 - f1_score: 0.4091
Epoch 2/10
3098/3098 [==============================] - 525s 169ms/step - loss: 0.6040 - binary_accuracy: 0.6728 - f1_score: 0.5035
Epoch 3/10
3098/3098 [==============================] - 496s 160ms/step - loss: 0.5926 - binary_accuracy: 0.6836 - f1_score: 0.5336
Epoch 4/10
3098/3098 [==============================] - 493s 159ms/step - loss: 0.5874 - binary_accuracy: 0.6880 - f1_score: 0.5480
Epoch 5/10
3098/3098 [==============================] - 495s 160ms/step - loss: 0.5836 - binary_accuracy: 0.6914 - f1_score: 0.5562
Epoch 6/10
3098/3098 [==============================] - 491s 159ms/step - loss: 0.5804 - binary_accuracy: 0.6935 - f1_score: 0.5630
Epoch 7/10
3098/3098 [==============================] - 489s 158ms/step - loss: 0.5785 - binary_accuracy: 0.6959 - f1_score: 0.5663
Epoch 8/10
3098/3098 [==============================] - 491s 158ms/step - lo

## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 49564 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

775/775 [==============================] - 28s 36ms/step - loss: 0.6275 - binary_accuracy: 0.6641 - f1_score: 0.4395


[0.627495288848877, 0.6640908718109131, 0.439519464969635]